In [20]:
import json
import numpy as np
import math
import cv2
import mediapipe as mp
from tensorflow.keras.models import load_model
from PIL import Image

In [35]:
# Define angle thresholds for each pose
ANGLE_THRESHOLDS = {
    'downdog': {
        'left_elbow_angle': (150, 200),
        'right_elbow_angle': (150, 200),
        'left_shoulder_angle': (150, 220),
        'right_shoulder_angle': (150, 220),
        'left_knee_angle': (140, 200),
        'right_knee_angle': (140, 200),
        'left_hip_angle': (60, 110),
        'right_hip_angle': (60, 110)
    },
    'goddess': {
        'left_elbow_angle': (70, 120),
        'right_elbow_angle': (70, 120),
        'left_shoulder_angle': (70, 110),
        'right_shoulder_angle': (70, 110),
        'left_knee_angle': (90, 140),
        'right_knee_angle': (90, 140),
        'left_hip_angle': (40, 110),
        'right_hip_angle': (40, 110)
    },
    'plank': {
        'left_elbow_angle': (150, 180),
        'right_elbow_angle': (150, 180),
        'left_shoulder_angle': (65, 90),
        'right_shoulder_angle': (65, 90),
        'left_knee_angle': (160, 180),
        'right_knee_angle': (160, 180),
        'left_hip_angle': (150, 180),
        'right_hip_angle': (150, 180)
    },
    'warrior2': {
        'left_elbow_angle': (160, 180),
        'right_elbow_angle': (160, 180),
        'left_shoulder_angle': (80, 120),
        'right_shoulder_angle': (80, 120),
        'left_knee_angle': (80, 120),
        'right_knee_angle': (80, 120),
        'left_hip_angle': (80, 120),
        'right_hip_angle': (80, 120)
    },
    'tree': {
        'left_elbow_angle': (160, 180),
        'right_elbow_angle': (160, 180),
        'left_shoulder_angle': (160, 185),
        'right_shoulder_angle': (160, 185),
        'left_knee_angle': (20, 55),
        'right_knee_angle': (170, 185),
        'left_hip_angle': (90, 130),
        'right_hip_angle': (160, 180)
    }
}


In [37]:
def calculate_average_angles(image_path, iterations=5):
    angles_list = []
    for _ in range(iterations):
        angles = extract_angles_from_image(image_path)
        if angles:
            angles_list.append(angles)
    if angles_list:
        averaged_angles = {k: np.mean([a[k] for a in angles_list]) for k in angles_list[0]}
        return averaged_angles
    else:
        print("No angles detected after multiple attempts.")
        return None

In [38]:
# Function to extract joint coordinates from the image and calculate angles
def extract_angles_from_image(image_path, return_landmarks=False):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    
    # Load and process image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        
        # Extract coordinates for key joints
        def get_coords(landmark):
            return [landmark.x, landmark.y]

        left_shoulder = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER])
        right_shoulder = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER])
        left_elbow = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW])
        right_elbow = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW])
        left_wrist = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_WRIST])
        right_wrist = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST])
        left_hip = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_HIP])
        right_hip = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_HIP])
        left_knee = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_KNEE])
        right_knee = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE])
        left_ankle = get_coords(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE])
        right_ankle = get_coords(landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE])

        # Calculate angles
        angles = {
            'left_elbow_angle': calculate_angle(left_shoulder, left_elbow, left_wrist),
            'right_elbow_angle': calculate_angle(right_shoulder, right_elbow, right_wrist),
            'left_shoulder_angle': calculate_angle(left_elbow, left_shoulder, left_hip),
            'right_shoulder_angle': calculate_angle(right_elbow, right_shoulder, right_hip),
            'left_knee_angle': calculate_angle(left_hip, left_knee, left_ankle),
            'right_knee_angle': calculate_angle(right_hip, right_knee, right_ankle),
            'left_hip_angle': calculate_angle(left_shoulder, left_hip, left_knee),
            'right_hip_angle': calculate_angle(right_shoulder, right_hip, right_knee)
        }

        if return_landmarks:
            landmarks_dict = {
                'left_elbow_angle': (left_shoulder, left_elbow, left_wrist),
                'right_elbow_angle': (right_shoulder, right_elbow, right_wrist),
                'left_shoulder_angle': (left_elbow, left_shoulder, left_hip),
                'right_shoulder_angle': (right_elbow, right_shoulder, right_hip),
                'left_knee_angle': (left_hip, left_knee, left_ankle),
                'right_knee_angle': (right_hip, right_knee, right_ankle),
                'left_hip_angle': (left_shoulder, left_hip, left_knee),
                'right_hip_angle': (right_shoulder, right_hip, right_knee)
            }
            return angles, landmarks_dict
        else:
            return angles
    else:
        return None

In [39]:
# Function to preprocess the image (resize and flatten)
from PIL import Image
def preprocess_image_for_model(image_path, size=(64, 64)):
    try:
        # Load the image
        img = Image.open(image_path).convert('RGB')
        
        # Resize the image
        img = img.resize(size)
        
        # Convert to numpy array and flatten
        img_array = np.array(img).flatten()
        
        # Reshape for model input
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        
        return img_array
    except Exception as e:
        print(f"Error during preprocessing: {e}")
        return None


In [40]:
# Function to reset the model (reload the model)
def reset_model(model_path='verification_model_newdataset.h5'):
    global model
    try:
        model = load_model(model_path)  # Reload the model from the specified path
        print("Model reloaded successfully.")
    except Exception as e:
        print(f"Error during model reset: {e}")

# Load initial model
reset_model()

Model reloaded successfully.


In [41]:
def predict_pose_correctness(image_path, pose_label):
    # Load image and extract angles with landmarks
    image = cv2.imread(image_path)
    angles, landmarks = extract_angles_from_image(image_path, return_landmarks=True)
    
    if angles is None:
        return f"No pose landmarks detected for {pose_label}."

    # Reshape angles for LSTM model
    angles_array = np.array(list(angles.values())).reshape(1, 1, len(angles))

    # Predict correctness using the model
    prediction = model.predict(angles_array)
    correctness = prediction[0][0] > 0.5  # Binary classification threshold

    if correctness:
        return "yes"
    else:
        incorrect_parts = []
        incorrect_coords = []  # Store coordinates of incorrect angles

        # Check each angle against thresholds
        for angle_name, angle_value in angles.items():
            if angle_name in ANGLE_THRESHOLDS[pose_label]:  # Ensure it checks only specific pose thresholds
                min_threshold, max_threshold = ANGLE_THRESHOLDS[pose_label][angle_name]
                
                if pose_label == 'tree':
                    # Handle special conditions for Tree Pose
                    if angle_name == 'left_knee_angle':
                        # Allow right knee to compensate if left knee is out of range
                        if not (20 <= angle_value <= 55 or 170 <= angle_value <= 185):
                            incorrect_parts.append(f"{angle_name} is out of range ({angle_value:.2f}°)")
                            point_a, point_b, point_c = landmarks[angle_name]
                            incorrect_coords.append((point_a, point_b, point_c))
                    elif angle_name == 'right_knee_angle':
                        # Allow left knee to compensate if right knee is out of range
                        if not (20 <= angle_value <= 55 or 170 <= angle_value <= 185):
                            incorrect_parts.append(f"{angle_name} is out of range ({angle_value:.2f}°)")
                            point_a, point_b, point_c = landmarks[angle_name]
                            incorrect_coords.append((point_a, point_b, point_c))
                    elif angle_name == 'left_hip_angle':
                        # Check hip angles
                        if not (90 <= angle_value <= 130 or 160 <= angle_value <= 180):
                            incorrect_parts.append(f"{angle_name} is out of range ({angle_value:.2f}°)")
                            point_a, point_b, point_c = landmarks[angle_name]
                            incorrect_coords.append((point_a, point_b, point_c))
                    elif angle_name == 'right_hip_angle':
                        if not (90 <= angle_value <= 130 or 160 <= angle_value <= 180):
                            incorrect_parts.append(f"{angle_name} is out of range ({angle_value:.2f}°)")
                            point_a, point_b, point_c = landmarks[angle_name]
                            incorrect_coords.append((point_a, point_b, point_c))
                else:
                    # General case for other poses
                    if not (min_threshold <= angle_value <= max_threshold):
                        incorrect_parts.append(f"{angle_name} is out of range ({angle_value:.2f}°)")
                        point_a, point_b, point_c = landmarks[angle_name]
                        incorrect_coords.append((point_a, point_b, point_c))

        # Draw circles around incorrect angles in the image
        for (point_a, point_b, point_c) in incorrect_coords:
            # Convert normalized coordinates to pixel coordinates
            height, width, _ = image.shape
            pa = (int(point_a[0] * width), int(point_a[1] * height))
            pb = (int(point_b[0] * width), int(point_b[1] * height))
            pc = (int(point_c[0] * width), int(point_c[1] * height))
            
            # Draw circles on each of the points involved in the incorrect angle
            cv2.circle(image, pa, 5, (0, 0, 255), -1)
            cv2.circle(image, pb, 5, (0, 0, 255), -1)
            cv2.circle(image, pc, 5, (0, 0, 255), -1)

        # Save or display the annotated image as needed
        cv2.imwrite("output_with_incorrect_angles.jpg", image)

        return "no", incorrect_parts


In [42]:
# Example usage
image_path = 'C:/Users/HP/Desktop/Dataset/Plank/00000053.jpg'
pose_label = 'plank'
correctness = predict_pose_correctness(image_path, pose_label)
print(correctness)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
('no', ['left_knee_angle is out of range (155.04°)', 'right_knee_angle is out of range (121.60°)', 'left_hip_angle is out of range (133.61°)', 'right_hip_angle is out of range (66.47°)'])
